Run this to make sure you have everything you need installed

In [1]:
# @title
# Install requirements
# This is run just to make sure we have enterything we need.
!pip install requests
!pip install bs4
!pip install pandas>=0.25.3
!pip install Pillow
!pip install plotly
!pip install chart_studio
!pip install wget
!pip install httpimport

**Import Python requirments:**

Most importantly:
[zp_tools](https://github.com/Heteroskedastic/Zwiftpower_tools) from github

In [3]:
%load_ext autoreload
%autoreload 2
import requests
import httpimport
import pandas as pd
# from google.colab import files
from datetime import date
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objects as go
import time
# this imports the code from github.
with httpimport.remote_repo(['zp_tools'], 'https://raw.githubusercontent.com/Heteroskedastic/Zwiftpower_tools/master'):
  import zp_tools as zpt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Get a list of all team with team data**
- Export and save as CSV localy

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
s=requests.Session()
zp_team_list = zpt.get_team_list(headers=headers, s=requests.Session())
zp_team_list.to_csv(f'zp_team_list_{date.today()}.csv')
#files.download(f'zp_team_list_{date.today()}.csv')

# Now we have the data lets explore

**Lets get some basic info about teams**

In [6]:
print('\nTotal teams')
print(len(zp_team_list))

print('\nHow many have no members?')
print(len(zp_team_list.riders[zp_team_list.riders == 0]))

print('\nHow many teams have 2 or less riders (not really teams)?')
print(len(zp_team_list.riders[zp_team_list.riders <= 2]))

print('\nHow many teams have 3 or more riders?')
print(len(zp_team_list.riders[zp_team_list.riders >= 3]))

pd.options.display.float_format = '{:,.2f}'.format
zp_team_list.sort_values(by='riders', ascending=False, inplace=True)
print('\nThe 10 largest teams and their racers to rider ratio')
zp_team_list.style.hide_index()
print(zp_team_list[['tln', 'riders', 'races_rider_ratio']].head(10).to_string(index=False))

print('\nThe 10 teams and with the highest race to rider ratio')
zp_team_list.sort_values(by='races_rider_ratio', ascending=False, inplace=True)
print(zp_team_list[zp_team_list.riders >= 3][['tln', 'riders', 'races_rider_ratio']].head(10).to_string(index=False))


Total teams
3296

How many have no members?
210

How many teams have 2 or less riders (not really teams)?
2062

How many teams have 3 or more riders?
1234

The 10 largest teams and their racers to rider ratio
               tln  riders  races_rider_ratio
 ZTPL Cycling Club    2040              21.95
               ZRG    1788              31.98
              DIRT    1422              60.11
  DanishBikeRiders     976              41.72
        Team Italy     967              22.00
          The Herd     712              48.41
           Vikings     652              49.93
     Belgian Zwift     630              41.79
    Bolt Race Team     568              87.64
     Team CRYO-GEN     551              86.26

The 10 teams and with the highest race to rider ratio
                      tln  riders  races_rider_ratio
                  OneLove       3             405.67
                      FTL       5             326.60
             TEAM IGNITER       8             323.62
        Indoor Sp

In [8]:
zp_team_list.sort_values(by='riders', ascending=True, inplace=True)
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=zp_team_list.riders[zp_team_list.riders >= 3]
))

fig.update_layout(
    title="Distribution of Team size",
    xaxis_title="Team size rank",
    yaxis_title="Member Count",
    yaxis_type="log",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
))

fig.show()

In [19]:
import time
real_teams = zp_team_list[zp_team_list.riders >= 3].copy()
real_teams.sort_values(by='riders', ascending=False, inplace=True)

c= ['team_id', 'div', 'divw', 'flag', 'r', 'ftp', 'name', 'aid', 'age', 'zada', 'w',
       'rank', 'skill', 'distance', 'climbed', 'energy', 'time', 'skill_race',
       'skill_seg', 'skill_power', 'zwid', 'status', 'reg', 'email',
       'h_1200_watts', 'h_1200_wkg', 'h_15_watts', 'h_15_wkg']
team_data = pd.DataFrame(columns = c)

count = 0
for t in real_teams.team_id:
  fail_count = 2
  try:
    # Be nice to zwift power
    td = zpt.get_team_data(t, headers=headers, s=s)
    td['team_id'] = t
    team_data = team_data.append(td, ignore_index=True, sort=False)
    # Be nice to zwift power
    time.sleep(.75)
  except Exception as e:
    print(f'Error on t')
    print(e)
    fail_count += 2
    time.sleep(fail_count)
  if fail_count >= 13:
    print(f'Giving up at team_id: {t}')
    continue
  count += 1
  if not count % 50:
    print(f'The {count} record on team_id: {t}')


print(f'Total records: {team_data.zwid.count()}\nUnique zwids: {team_data.zwid.nunique()}')
team_data.to_csv(f'data/team_data_{date.today()}.csv')
#files.download(f'data/team_data_{date.today()}.csv')

The 50 record on team_id: 1940
The 100 record on team_id: 4494
The 150 record on team_id: 189
The 200 record on team_id: 2783
The 250 record on team_id: 2894
The 300 record on team_id: 2279
The 350 record on team_id: 2687
The 400 record on team_id: 4862
The 450 record on team_id: 2790
The 500 record on team_id: 4929
The 550 record on team_id: 3110
The 600 record on team_id: 5218
The 650 record on team_id: 3482
The 700 record on team_id: 3044
The 750 record on team_id: 3049
The 800 record on team_id: 4405
The 850 record on team_id: 3191
The 900 record on team_id: 2175
The 950 record on team_id: 2152
The 1000 record on team_id: 2156
The 1050 record on team_id: 1915
The 1100 record on team_id: 2982
The 1150 record on team_id: 5255
The 1200 record on team_id: 2801
Total records: 29167
Unique zwids: 27758


In [34]:
team_data['r'] = team_data['r'].replace({'': 0})
team_data = team_data.astype({'r': 'int32'})
team_data.sort_values(by='team_id', ascending=True, inplace=True)
team_data.reset_index(drop=True)
team_data.to_csv(f'zp_team_riders_{date.today()}.csv')
#files.download(f'zp_team_riders_{date.today()}.csv')


In [36]:
team_data.sort_values(by='r', ascending=True, inplace=True)
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=team_data.r
))
fig.update_layout(
    title="Distribution of rider races",
    xaxis_title="Rider",
    yaxis_title="Race Count",
    yaxis_type="log",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
))
fig.show()

In [40]:


team_data.sort_values(by='r', ascending=True, inplace=True)
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=team_data.r[team_data.team_id=='2740']
))
fig.update_layout(
    title="Distribution of rider races",
    xaxis_title="Rider",
    yaxis_title="Race Count",
    yaxis_type="log",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
))
fig.show()

In [4]:
zp_team_list

,tln,tag,team_id,riders,races,color,contact,name,aid,tid,tname,tc,tbc,tbd,flag,primary,races_rider_ratio
11,ZTPL Cycling Club,ZTPL.CC,25,2053,45160,ffffff,https://www.facebook.com/ZwiftPolska,Jacek Tarach,62218,,,,,,pl,0,21.997077
389,ZRG,ZRG,1942,1797,57493,1bc1f1,https://www.facebook.com/groups/Zwiftridersger...,Stephan Picker (ZRG-R),30728,,,,,,de,0,31.993879
1037,DIRT,DIRT,2707,1430,86150,29ec34,https://www.facebook.com/groups/ZWIFTDIRT,Jason Sterny (DIRT),413819,,,,,,us,0,60.244755
432,DanishBikeRiders,DBR,1986,985,40940,ffffff,https://www.facebook.com/groups/1160541327367604/,Søren Andersen,176479,,,,,,dk,0,41.563452
141,Team Italy,Team Italy,588,972,21470,dd4b39,https://www.facebook.com/groups/zwiftitaly/,Andrea Anastasio,182210,,,,,,it,0,22.088477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,Wheelsuckers,WSUCK,2115,0,0,ffffff,,Jens Bernhardt,277152,,,,,,de,0,NaN
3167,SÃ¶dertÃ¤lje CK,SCK,5375,0,0,ffffff,,Björn Önnemar,986610,,,,,,se,0,NaN
2398,Igniter,IGN,4572,0,0,ffffff,,Russ Cripps,166888,,,,,,gb,0,NaN
1567,Scissorbird,Scissorbird,3386,0,0,ffffff,,Andy Kroll (R3R Westlife),993035,,,,,,us,0,NaN


In [5]:
zp_team_list.columns

Index(['tln', 'tag', 'team_id', 'riders', 'races', 'color', 'contact', 'name',
       'aid', 'tid', 'tname', 'tc', 'tbc', 'tbd', 'flag', 'primary',
       'races_rider_ratio'],
      dtype='object')

In [12]:
zp_team_list.tbc.unique()

array([''], dtype=object)